In [2]:
import sys
import gi
import logging

gi.require_version("GLib", "2.0")
gi.require_version("GObject", "2.0")
gi.require_version("Gst", "1.0")

from gi.repository import Gst, GLib, GObject

logging.basicConfig(level=logging.DEBUG, format="[%(name)s] [%(levelname)8s] - %(message)s")
logger = logging.getLogger(__name__)

In [5]:
logging.DEBUG #10
logger #Hataları vs. yazdırmaya yarıyor

<Logger __main__ (DEBUG)>

In [14]:
Gst.init(sys.argv[1:])

source=Gst.ElementFactory.make('videotestsrc','source')
sink=Gst.ElementFactory.make('autovideosink','sink')

pipeline=Gst.Pipeline.new('test-pipeline')

if not source or not sink or not pipeline:
    logger.error('Elementlerde sıkıntı var...')
    sys.exit(1)

pipeline.add(source)
pipeline.add(sink)

if source.link(sink):
    logger.info('Bağlandı')

source.set_property('pattern', 2) #source.props.pattern = 0

ret=pipeline.set_state(Gst.State.PLAYING)

if ret==Gst.StateChangeReturn.FAILURE:
    logger.error("pipeline'da sıkıntı var")
    sys.exit(1)

bus = pipeline.get_bus()
msg = bus.timed_pop_filtered(
    Gst.CLOCK_TIME_NONE, 
    Gst.MessageType.ERROR | Gst.MessageType.EOS)

if msg:
    if msg.type == Gst.MessageType.ERROR:
        err, debug_info = msg.parse_error()
        logger.error(f"Error received from element {msg.src.get_name()}: {err.message}")
        logger.error(f"Debugging information: {debug_info if debug_info else 'none'}")

    elif msg.type == Gst.MessageType.EOS:
        logger.info("End-Of-Stream reached.")

    else:
        # This should not happen as we only asked for ERRORs and EOS
        logger.error("Unexpected message received.")

pipeline.set_state(Gst.State.NULL)

[__main__] [    INFO] - Bağlandı
[__main__] [   ERROR] - Error received from element sink-actual-sink-xvimage: Output window was closed
[__main__] [   ERROR] - Debugging information: xvimagesink.c(554): gst_xv_image_sink_handle_xevents (): /GstPipeline:test-pipeline/GstAutoVideoSink:sink/GstXvImageSink:sink-actual-sink-xvimage


<Gst.Message object at 0x7f88001ff6a0 (GstMessage at 0x7f87c8002ac0)>


<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>

In [11]:
source.get_property('pattern') # değiştirip çalıştırınca değişik görüntüler çıkıyor

<enum SMPTE 100% color bars of type __main__.GstVideoTestSrcPattern>

# Açıklamalar

All elements in GStreamer must typically be contained inside a pipeline before they can be used, because it takes care of some clocking and messaging functions. We create the pipeline with Gst.Pipeline.new().

A pipeline is a particular type of Gst.Bin, which is the element used to contain other elements. Therefore all methods which apply to bins also apply to pipelines.

n our case, we call Gst.Bin.add() to add elements to the pipeline. The function accepts any number of Gst Elements as its arguments

These elements, however, are not linked with each other yet. For this, we need to use Gst.Element.link(). Its first parameter is the source, and the second one the destination. The order counts, because links must be established following the data flow (this is, from source elements to sink elements). Keep in mind that only elements residing in the same bin can be linked together, so remember to add them to the pipeline before trying to link them!

GStreamer bus: It is the object responsible for delivering to the application the Gst.Messages generated by the elements, in order and to the application thread. This last point is important, because the actual streaming of media is done in another thread than the application.



# Alıştırma

If you feel like practicing, try this exercise: Add a video filter element in between the source and the sink of this pipeline. Use vertigotv for a nice effect. You will need to create it, add it to the pipeline, and link it with the other elements.

Depending on your platform and available plugins, you might get a “negotiation” error, because the sink does not understand what the filter is producing...

In [5]:
import sys
import gi
import logging

gi.require_version("GLib", "2.0")
gi.require_version("GObject", "2.0")
gi.require_version("Gst", "1.0")

from gi.repository import Gst, GLib, GObject

logging.basicConfig(level=logging.DEBUG, format="[%(name)s] [%(levelname)8s] - %(message)s")
logger = logging.getLogger(__name__)

Gst.init(sys.argv[1:])

pipeline2=Gst.Pipeline.new('exerc')

source=Gst.ElementFactory.make('videotestsrc','source')
vertigotv=Gst.ElementFactory.make('vertigotv','vertigo')
convert=Gst.ElementFactory.make('videoconvert','convert')
sink=Gst.ElementFactory.make('autovideosink','sink')

pipeline2.add(source)
pipeline2.add(vertigotv)
pipeline2.add(convert)
pipeline2.add(sink)

source.link(vertigotv)
vertigotv.link(convert)
convert.link(sink)

ret=pipeline2.set_state(Gst.State.PLAYING)
if ret==Gst.StateChangeReturn.FAILURE:
    logger.error("pipeline'da sıkıntı var")
    sys.exit(1)

bus=pipeline2.get_bus()
msg=bus.timed_pop_filtered(
    Gst.CLOCK_TIME_NONE,
    Gst.MessageType.ERROR | Gst.MessageType.EOS)


if msg:
    if msg.type == Gst.MessageType.ERROR:
        err, debug_info = msg.parse_error()
        logger.error(f"Error received from element {msg.src.get_name()}: {err.message}")
        logger.error(f"Debugging information: {debug_info if debug_info else 'none'}")

    elif msg.type == Gst.MessageType.EOS:
        logger.info("End-Of-Stream reached.")

    else:
        # This should not happen as we only asked for ERRORs and EOS
        logger.error("Unexpected message received.")

pipeline2.set_state(Gst.State.NULL)


[__main__] [   ERROR] - Error received from element sink-actual-sink-xvimage: Output window was closed
[__main__] [   ERROR] - Debugging information: xvimagesink.c(554): gst_xv_image_sink_handle_xevents (): /GstPipeline:exerc/GstAutoVideoSink:sink/GstXvImageSink:sink-actual-sink-xvimage


<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>